# Linear Regression

In this notebook, we implement Linear Regression from scratch.
The implementation is applied onto a synthetic dataset and fit by two methods - Gradient Descent and Normal Equations.

Before we begin, let us import the necessary libraries:

- `numpy`, the basic library for scientific computing in Python
- `numpy.typing`, for type hints.
- `sklearn`, which we primarily use for generating synthetic linear regression data.

In [1]:
import numpy as np
import numpy.typing as npt
import sklearn.linear_model
import sklearn.datasets

## Introduction

A **regression problem** is a type of supervised learning problem in which we map input vectors $\textbf{x}$ to outputs $y$. While in general these outputs may themselves be vectors (something known as **multivariate regression**), we shall assume that $y \in \mathbb{R}$, that is, y is just a single real number for each vector $\textbf{x}$. 

In general, we may have $N$ total $(\textbf{x}, y)$ pairs, which together we call the **dataset** for the problem. The elements of $\textbf{x}$ are called the **features** and $y$ is called the **target** variable.

The basic assumption of **Linear Regression** ( also called its *inductive bias* ) is that the relationship between $\textbf{x}$ and $y$ is **linear**. That is, the underlying function that we are trying to model exists as a linear function: 

$$ y = f_{\textbf{w}, b}(\textbf{x}) = \textbf{w}^{T}\textbf{x} + b $$

Where $\textbf{w}, b$ are the *parameters* of the model.

In [2]:
def f(x: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float) -> float:
    f_wb: float = np.dot(w, x) + b
    return f_wb

### The Cost Function

To fit the model to the data, we need a measure of how good the fit is. Such a measure is often provided by a real-valued function called a **loss function** or **cost function**.

It is common to use the **Squared Error Cost Function** for regression problems, given by 

$$J(\textbf{w}, b) = \frac{1}{2N}\sum_{i=1}^{N} \left(y^{(i)} - f_{\textbf{w}, b}(\textbf{x}^{(i)})\right)^{2}$$

This cost function evaluates, on average, how far the predictions made by the model (for a particular choice of parameters) is from the known targets. Here the superscript $i$ indicates that the features and targets belong to example $i$ in the dataset.

Note: To be exact, there is a slight difference between the **loss function** and the **cost function**. While the terms may be used interchangibly in informal discussions, the **loss** function usually refers to the loss incurred on **one** training example, while the **cost** function refers to the loss incurred, on average, over the whole dataset.

In [3]:
def cost(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float) -> float:
    N : int = len(y)
    C : float = 0.0
    for i in range(N):
        C = C + (y[i] - f(X[i], w, b))**2.0
    C = C/(2.0*N)
    return C

Naturally, then, the parameters of the model are found by minimizing the cost function. 

For linear regression, this can be done analytically (shown below), or it can be done by an iterative method known as Gradient Descent. 

Gradient Descent is the fundamental method behind training complex models such as neural networks, and is also described below briefly.

### Analytical Solution to Linear Regression

The cost function for Linear Regression can be written as:
$$ J(\textbf{w}, b) = \frac{1}{2N}(\textbf{y} - \textbf{X}\beta)^{T}(\textbf{y} - \textbf{X}\beta)$$

Where $\textbf{X}$ is a $N \times (p + 1)$ matrix, $p$ denoting the dimension of each input, $\textbf{x}$. 

We add an extra column of all ones to $\textbf{X}$, to absorb the bias, $b$, into $\beta$. 

$\beta$ is a $(p + 1) \times 1$ dimensional vector consisting of $\textbf{w}$ and $b$ stacked on top of each other.

$\textbf{y}$ denotes the $N \times 1$ vector denoting the output labels.


Differentiating this function with respect to $\beta$ gives:

$$ \dfrac{\partial J}{\partial \beta} = \frac{1}{2N} \left((\textbf{X}^T \textbf{X})\beta - \textbf{X}^T \textbf{y}\right)$$

and doing it a second time gives:

$$ \dfrac{\partial^2 J}{\partial \beta^2} = \frac{1}{2N}\textbf{X}^T\textbf{X}$$

If $\textbf{X}$ is full-rank (that is, all the features are independent of each other), then $\textbf{X}^T\textbf{X}$ is positive-definite. 

In such a case, the optimum found by setting the first derivative to zero is the minima of the cost function, and the value of $\beta$ at the minima is:

$$ \beta = \left((\textbf{X}^T\textbf{X})^{-1}\textbf{X}^T\right)\textbf{y}$$

This method of solution is called solution by **normal equations**. Unfortunately, it breaks down if two features are correlated with each other, a problem known as **collinearity** or **multicollinearity**.

In [4]:
def normal_solution(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
    beta : npt.NDArray[np.float64] = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X)), X.T), y)
    return beta

### Gradient Descent-based solution

Another method to find the parameters is to use **Gradient Descent** to minimize the loss function. This is an iterative method, and it is based on the result that the gradient of a function $f$ (denoted $\nabla f$) is always along the direction of steepest increase of $f$. As a result, taking small steps directly opposite to the direction of the gradient will decrease the function slowly till we eventually reach the minimum (where the gradient is zero).

This method in general only finds the local minimum. However, for linear regression, since the problem can be formulaed as a **convex optimization** problem (the cost function $J(\textbf{w}, b)$ is convex), the local minimum **is** the global minimum. Hence, we find the same solution as by the normal equations method.

More formally, gradient descent will perform the following updates till convergence:

$$ \textbf{w} \leftarrow \textbf{w} - \eta \nabla_{\textbf{w}} f $$
$$ b \leftarrow b - \eta \nabla_{b} f $$

Where $\eta$ is a small positive quantity called the **learning rate**. It is the step size for the updates.

For $f = J(\textbf{w}, b)$ (that is, for linear regression), the gradients can be found analytically:

$$\nabla_{\textbf{w}} J(\textbf{w}, b) = -\frac{1}{N}\sum_{i = 1}^{N} \left( y^{(i)} - f_{\textbf{w}, b}(\textbf{x}^{(i)})\right)\textbf{x}^{(i)} $$
$$\nabla_{b} J(\textbf{w}, b) = -\frac{1}{N}\sum_{i = 1}^{N} \left( y^{(i)} - f_{\textbf{w}, b}(\textbf{x}^{(i)})\right) $$

It should be noted that the above equations define what is called **batch** gradient descent, since we use the entire dataset to estimate the gradients before updating the parameters. Another extreme is to use a single point to estimate the gradients, update the parameters, and then re-estimate the gradients for the next point, repeating till convergence (and looping back to the first point after the entire dataset has been considered). This is called **stochastic gradient descent**. 

A method between these two is called **mini-batch gradient descent**, where a set of training examples, $m < N$ in number, are chosen to estimate the gradient.

Stochastic gradient descent and mini-batch gradient descent aren't really necessary for linear regression, but are often used in other optimization problems where either each training example takes up a large amount of memory, or there are too many training examples.

In [5]:
def grad_cost(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float) -> tuple[npt.NDArray[np.float64], float]:
    N: int = len(y)
    dJ_dw: npt.NDArray[np.float64] = np.zeros(X.shape[-1])
    dJ_db: float = 0.0
    for i in range(N):
        dJ_dw = dJ_dw + (-1.0/N)*(y[i] - f(X[i], w, b))*X[i]
        dJ_db = dJ_db + (-1.0/N)*(y[i] - f(X[i],w,b))
    return (dJ_dw, dJ_db)

def gd_step(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float, eta: float) -> tuple[npt.NDArray[np.float64], float]:
    dJ_dw, dJ_db = grad_cost(X, y, w, b)
    w_new: npt.NDArray[np.float64]  = w - eta * dJ_dw
    b_new: float = b - eta * dJ_db 
    return (w_new, b_new)
    
    
def gd_solution(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float, eta: float, eps: float) -> tuple[npt.NDArray[np.float64], float]:
    max_iter : int = 1_000_000 # stop if solution hasn't converged till 1,000,000th step.
    flag: bool = False # flag turns true if converged.

    w0: npt.NDArray[np.float64] = np.copy(w)
    b0: float = np.copy(b)

    cost_0 = cost(X, y, w0, b0)

    for i in range(max_iter):
        w0, b0 = gd_step(X, y, w0, b0, eta)
        cost_i = cost(X, y, w0, b0)
        
        if (np.abs(cost_i - cost_0) < eps): # check for convergence
            flag = True
            break
        else:
            cost_0 = cost_i 
    
    if (flag == True):
        print("Convergence achieved")
    else:
        raise ValueError(f"Failed to converge after {max_iter} iterations.")
    
    return (w0, b0)
        

## Regularization

Regularization is, in general, a technique to control model complexity. Models with large regularization have lower complexity relative to models with small regularization (or none at all). This has to do with the bias-variance tradeoff - regularization is a method to tune the **bias** of a model.

Since linear regression is a rather simple model, in many cases, regularization may not be strictly necessary. However, besides managing complexity, it also enforces extra constraints on the parameters, which can be helpful in certain cases, such as:

- When there are too many parameters but not enough training examples (high-dimensional linear models).
- When some features are correlated with each other.

A specific kind of regularization, called L1 regularization (or Lasso), encourages sparsity in parameters and can act as a feature selector as well.

### Ridge and Lasso Regression

Ridge and Lasso regression are just two of the most common ways of adding regularization to linear regression.

- **Ridge** regression uses something called **L2 regularization** where the cost function is penalized based on the $L_2$ norm of the weight vector, $\textbf{w}$:

    $$ J(\textbf{w}, b) = \frac{1}{2N}\sum_{i=1}^{N} \left(y^{(i)} - f_{\textbf{w}, b}(\textbf{x}^{(i)})\right)^{2} + \frac{\lambda}{N}||\textbf{w}||^2$$

    $\lambda$ is a tunable parameter called the **regularization strength**. It can be shown that this is equivalent to imposing a constraint on the norm of the weight vector (that is, choosing a $t$ such that $||\textbf{w}||^2 \leq t$). As one might expect, Ridge regression tends to shrink the weights. It also has the advantage of being analytically simple and leading to closed-form solutions, as the $L_2$ norm is differentiable.

    

- **Lasso** Regression uses something called **L1 regularization**, where the cost function is penalized based on the $L_1$ norm of the weight vector (i.e., the sum of the absolute values of the components of the vector). 

    L1 regularization tends to promote sparsity in the solution vector, i.e., it prefers to set some components to zero. As a result, the components that are left nonzero can be considered as the ones that are important to the model, while the rest (the extraneous features) are essentially dropped by the model.
    
    The loss function for Lasso Regression is given by:

    $$ J(\textbf{w}, b) = \frac{1}{2N}\sum_{i=1}^{N} \left(y^{(i)} - f_{\textbf{w}, b}(\textbf{x}^{(i)})\right)^{2} + \frac{\lambda}{N}\left(\sum_{i=1}^{p}|w_i|\right)$$ 

    where $p$ is the number of features for each training example, and the components $w_i$ are called the *feature weights*.


For an intutive understanding of the difference between the two regularizations, consider an example where the loss may be reduced by either setting $100 \rightarrow 99.1$ for one feature weight, or setting $1 \rightarrow 0$ for another feature weight. 
    
$L_2$ regularization would prefer the former, as the difference in squares is larger, and so the decrease in cost is larger. 
    
$L_1$ regularization would instead prefer the latter, as the absolute decrease in cost is larger in the latter. Consequently, we can see why $L_1$ regularization would promote sparsity.


In [6]:
def ridge_cost(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float, lambd: float) -> float:
    N : int = len(y)
    C : float = 0.0
    for i in range(N):
        C = C + (y[i] - f(X[i], w, b))**2.0
    C = C/(2.0*N)
    C = C + (lambd/N)*(np.linalg.norm(w, ord=2)**2.0) # L2 regularization
    return C

def lasso_cost(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float, lambd: float) -> float:
    N : int = len(y)
    C : float = 0.0
    for i in range(N):
        C = C + (y[i] - f(X[i], w, b))**2.0
    C = C/(2.0*N)
    C = C + (lambd/N)*np.linalg.norm(w, ord=1) # L1 regularization
    return C


def reg_grad_cost(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float, lambd : float, regularization : str = "ridge") -> tuple[npt.NDArray[np.float64], float]:
    N: int = len(y)
    dJ_dw: npt.NDArray[np.float64] = np.zeros(X.shape[-1])
    dJ_db: float = 0.0
    if regularization == "ridge":
        for i in range(N):
            dJ_dw = dJ_dw + (-1.0/N)*(y[i] - f(X[i], w, b))*X[i] + (2.0*lambd/N)*w
            dJ_db = dJ_db + (-1.0/N)*(y[i] - f(X[i],w,b))
        return (dJ_dw, dJ_db)
    elif regularization == "lasso":
        for i in range(N):
            dJ_dw = dJ_dw + (-1.0/N)*(y[i] - f(X[i], w, b))*X[i] + (lambd/N)*np.sign(w)
            dJ_db = dJ_db + (-1.0/N)*(y[i] - f(X[i],w,b))
        return (dJ_dw, dJ_db)
    else:
        raise ValueError("Parameter `regularization` did not recieve expected input")


def reg_gd_step(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float, lambd : float, regularization : str,  eta: float) -> tuple[npt.NDArray[np.float64], float]:
    dJ_dw, dJ_db = reg_grad_cost(X, y, w, b, lambd, regularization)
    w_new: npt.NDArray[np.float64]  = w - eta * dJ_dw
    b_new: float = b - eta * dJ_db 
    return (w_new, b_new)

def reg_gd_solution(X: npt.NDArray[np.float64], y: npt.NDArray[np.float64], w: npt.NDArray[np.float64], b: float, lambd: float, regularization: str, eta: float, eps: float) -> tuple[npt.NDArray[np.float64], float]:
    max_iter : int = 1_000_000 # stop if solution hasn't converged till 1,000,000th step.
    flag: bool = False # flag turns true if converged.

    w0: npt.NDArray[np.float64] = np.copy(w)
    b0: float = np.copy(b)
    
    if regularization=="ridge":
        local_cost = ridge_cost
    elif regularization=="lasso":
        local_cost = lasso_cost
    else:
        raise ValueError("Parameter `regularization` did not recieve expected input")

    cost_0 : float = local_cost(X, y, w0, b0, lambd)

    for i in range(max_iter):
        w0, b0 = reg_gd_step(X, y, w0, b0, lambd, regularization, eta)
        cost_i : float = local_cost(X, y, w0, b0, lambd)
        
        if (np.abs(cost_i - cost_0) < eps): # check for convergence
            flag = True
            break
        else:
            cost_0 = cost_i 
    
    if (flag == True):
        print("Convergence achieved")
    else:
        raise ValueError(f"Failed to converge after {max_iter} iterations.")
    return (w0, b0)


## Running on Simulated Dataset

In [7]:
def generate_data(n_samples : int, n_features : int, collinear : bool = False, n_collinear : int = 0, corr_strength : float = 0.9, noise : float = 1.0):
    '''
    Wrapper to generate data for regression.
    '''
    X, y, coef = sklearn.datasets.make_regression(n_samples = n_samples, n_features=n_features,
                                 n_informative=n_features - n_collinear, n_targets=1, 
                                 bias=2.0, effective_rank=n_features - n_collinear,
                                 noise=noise, shuffle=True, random_state=42, coef=True)
    y : npt.NDArray[np.float64] = y.reshape(-1, 1)
    if (collinear==True):
        for i in range(n_features - n_collinear, n_features):
            X[:, i] = corr_strength * X[:,np.random.randint(0, n_features - n_collinear)] + (1 - corr_strength) * X[:, i] # introduce strong collinearity
    return X, y, coef

### Preprocessing for Linear Regression

In real datasets, we often need to perform some sort of preprocessing or cleanup before applying linear regression. The most common steps involved are:



##### 1. Dealing with NaN (missing) values
    
Sometimes the dataset may just have missing values. There is a plethora of ways for dealing with missing values.

If, for instance, a certain feature is mostly missing values, then it may make sense to drop it - we may not be able to get much information from it after all.

Similarly, if a single training example has too many missing values, then we may want to drop that example as well - it is unlikely to be very informative.

On the other hand, if a small number of values are missing, we may use methods of varying complexity, such as:

- Set the missing values to be equal to the mean or median of the value of the feature in different training examples.
- Build a probability distribution over the feature, and draw from it to generate many different datasets.
- Use k-NN to average over only the k most similar training examples.

Alternatively, in some cases, the very fact that the value is absent may be useful. We may, thus, treat NaNs just as another value that the feature can take.


##### 2. Dealing with categorical attributes.

Although the discussions so far have involved us assuming that x is a vector quantity (more precisely, $x\in \mathbb{R}^p$), in general, some of the features may be categorical rather than continuous.

One way to deal with such features (or attributes) is to find an appropriate numeric **encoding**. 

- If the categorical feature has some inherent ordering, we may be able to use that ordering to assign values such as $1, 2, 3$ and so on to the $k$ possible categorical values. This is called **Ordinal Encoding**.

- If there is no ordering, we can split the categorical feature (which can take $k$ possible values) into $k$ binary features, denoting the presence or absence of the corresponding value. In practice, one of the features is often dropped, since if all the remaining $k - 1$ features are zero (or False), then naturally that denotes that the dropped feature must have a value of $1$ (or True). More formally, the feature is dropped since it is correlated to the remaining $k-1$ features, and this can lead to issues with multicollinearity.

While one-hot encoding is useful, it can cause the dataset to become rather high-dimensional. As a result, we may run into issues common with high dimensional data (such as data sparsity and the consequent susceptibility to overfitting). Regularization can often help in such cases, as we've discussed earlier.




##### 3. Standardization.

It is common practice to transform the data so that it has mean $0$ and variance $1$. This can be done by estimating the mean and variance of the input $\textbf{X}$ and $\textbf{y}$, and then subtracting the mean and dividing by the variance:
$$ \textbf{X} \leftarrow (\textbf{X} - \mu_{\textbf{X}})/\sigma^2_{\textbf{X}}$$
$$ \textbf{y} \leftarrow (\textbf{y} - \mu_{\textbf{y}})/\sigma^2_{\textbf{y}} $$

Where 
$$\mu_{\textbf{X}} = \frac{1}{N}\sum_{i}\textbf{x}^{(i)}$$
$$\mu_{\textbf{y}} = \frac{1}{N}\sum_{i} y^{(i)}$$
$$\sigma^2_{\textbf{X}} = \frac{1}{N-1}\sum_{i}(\textbf{x}^{(i)} - \mu_{\textbf{X}})^2$$
$$\sigma^2_{\textbf{y}} = \frac{1}{N - 1} \sum_{i} (y^{(i)} - \mu_{\textbf{y}})^2 $$

When we use our model to make predictions on unseen data (data not in the training set), we must be careful to also preprocess it identically, using the same parameters as the training data.
This is because we expect the training data to be representative of the underlying data distribution, and so these estimates of the mean and variance are actually estimates of the **true** mean and variance of the data distribution.


In [8]:
def standardize(X : npt.NDArray[np.float64], y : npt.NDArray[np.float64], train_set : bool = False, params : tuple[npt.NDArray[np.float64], npt.NDArray[np.float64], npt.NDArray[np.float64], npt.NDArray[np.float64]] = None) -> tuple[npt.NDArray[np.float64], npt.NDArray[np.float64], tuple[npt.NDArray[np.float64], npt.NDArray[np.float64], npt.NDArray[np.float64], npt.NDArray[np.float64]]]:

    if (train_set == True) and (params is None):
        mu_X : npt.NDArray[np.float64] = np.mean(X, axis=0)
        mu_y : npt.NDArray[np.float64] = np.mean(y)
        std_X : npt.NDArray[np.float64] = np.std(X, axis=0)
        std_y : npt.NDArray[np.float64] = np.std(y, axis=0)

        X : npt.NDArray[np.float64] = (X - mu_X)/std_X
        y : npt.NDArray[np.float64] = (y - mu_y)/std_y
        params : tuple[npt.NDArray[np.float64], npt.NDArray[np.float64], npt.NDArray[np.float64], npt.NDArray[np.float64]] = (mu_X, mu_y, std_X, std_y)
        
    elif (train_set == False) and (params is not None):
        mu_X, mu_y, std_X, std_y = params
        X : npt.NDArray[np.float64] = (X - mu_X)/std_X
        y : npt.NDArray[np.float64] = (y - mu_y)/std_y

    else:
        raise ValueError("Invalid set of inputs! Please ensure `params` is not None for train_set == False")
    
    return X, y, params



##### 4. Finding and dealing with multicollinearity.

If two or more features are correlated, it can negatively impact the convergence of linear regression - the normal equations method may fail as the Hessian may no longer be positive definite (and if it is, the condition number, a quantity describing the numerical instability of a matrix, may be too large). The Gradient Descent method will still work in some cases, but its convergence may become impacted and require a very small step size for each update.

Multicollinearity can also impact the interpretability of the model - if two or more features are correlated, we can no longer interpret their coefficients in terms of (relative) feature importances, since any change in one can be adjusted by appropriate changes in other(s).

Before we can take steps to deal with multicollinearity, we should have a measure of **how much** multicollinearity there is amongst features. This is done by a metric known as the **Variance Inflation Factor** (VIF).

The VIF for the $i$ th feature, $x_i$, can be computed by regressing the feature on the remaining features:

$$ VIF(x_i) = \frac{1}{1 - R_i^2} $$

Where the **coefficient of determiniation** ($R_i^2$) for the $i$ th feature is defined by:

$$ R_i^2 = 1 - \frac{\text{sum of squares of residuals}}{(N - 1) * (\text{variance of ith feature})}$$


A rule of thumb is that a VIF between $1$ (no multicollinearity at all) and $5$ is usually tolerable, but a higher VIF indicates significant multicollinearity and should be corrected.

There are many ways to deal with multicollinearity. One way would just be to drop the feature with high VIF. We aren't losing any information doing this either, since its information would be contained in the other features it is correlated to.

Alternatively, we could use other methods for regression, such as **partial least squares** (PLS) or **principal components regression** (PCR). Both of these methods essentially create a new set of features that are linearly independent. Both these methods will be explored in later notebooks.

In [9]:
def MaxVif(X : npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
    ''' Returns the Maximum VIF amongst all the features'''
    N, K = X.shape
    vif_i : list[float] = []
    for i in range(K):
        x_i : npt.NDArray[np.float64] = X[:, i].reshape(-1)
        x_rest : npt.NDArray[np.float64] = np.delete(X, i, axis=1)
        x_i_pred : npt.NDArray[np.float64] = sklearn.linear_model.LinearRegression().fit(x_rest, x_i).predict(x_rest)
        R_i_sq : float = 1 - np.sum(np.power(x_i - x_i_pred, 2.0))/(np.sum(np.power(x_i - np.mean(x_i), 2)))
        vif_i.append(1.0/(1.0 - R_i_sq))
    vif_i = np.array(vif_i)

    return np.array([np.max(vif_i), np.argmax(vif_i)])


##### 5. Performance Metrics for Linear Regression

Once we have fit the model, we are interested in knowing how **good** the fit is - how well is the regression model doing on seen, or unseen, data? 
A popular idea in machine learning in general is that of a **real-valued metric**, that is, a single real number that quantifies how good the model is. The following are some common metrics that are used in regression:

1. Mean Squared Error (MSE)

    The Mean Squared Error of the model, $f(x)$, on a dataset $X$ of $N$ samples (each denoted by the corresponding superscript $x^{(i)}$) with given labels or outputs $y$ (respectively, $y^{(i)}$) is:

    $$ \text{MSE}(f) = \frac{1}{N}\sum_{i=1}^{N}(y^{(i)} - f(x^{(i)}))^2$$

    This is just the same as the loss function we looked at above - but generalized for any dataset.

2. Mean Absolute Error (MAE)

    When the training dataset has a number of outliers, we may not want to heavily penalize the model for not being able to fit them. In such a case, the loss function (and evaluation metric) we use is the Mean Absolute Error:

    $$ \text{MAE}(f) = \frac{1}{N}\sum_{i=1}^{N}|y^{(i)} - f(x^{(i)})|$$

    We did not discuss this above since this is not very commonly used in practice, and it does not have a nice normal-form kind of solution (note that the absolute value function is not differentiable at $0$). Still, we can find a minima for this loss function using iterative methods.

3. $R^2$

    A common metric for linear regression is $R^2$, which, as defined above, is:

    $$ R^2 = 1 - \frac{\sum_i (y^{(i)} - f(x^{(i)}))^2}{\sum_i (y^{(i)} - \text{mean(y)})^2}$$

    It usually lies within $0$ (baseline model, always predicts output as the mean) and $1$ (predicts outputs perfectly). It is possible for $R^2 < 0$ if the model is **worse** than baseline.

In [10]:
def PerformanceSummary(y : npt.NDArray[np.float64], y_pred : npt.NDArray[np.float64]) -> dict[str, float]:
    y_bar : float = np.mean(y)
    mse_f : float = np.sum(np.power(y - y_pred, 2.0))/len(y)
    mae_f : float = np.sum(np.absolute(y - y_pred))/len(y)
    rsq : float = 1 - (np.sum(np.power((y - y_pred), 2.0)))/(np.sum(np.power((y - y_bar), 2.0)))
    perf : dict[str, float] = {"MSE":mse_f, "MAE": mae_f, "R^2": rsq}

    return perf

In [ ]:
def generate_datasets(n_samples : int , n_train : int, n_features : int = 10, collinear : bool = True, n_collinear : int = 2, corr_strength : float = 0.6, noise : float = 2.0) -> tuple[npt.NDArray[np.float64], npt.NDArray[np.float64], npt.NDArray[np.float64], npt.NDArray[np.float64]]:

    X, y, _ = generate_data(n_samples, n_features = n_features, collinear=collinear, n_collinear=n_collinear, corr_strength=corr_strength, noise = noise)

    X_train : npt.NDArray[np.float64] = X[:n_train]
    y_train : npt.NDArray[np.float64] = y[:n_train]
    X_train, y_train, params = standardize(X_train, y_train, train_set = True, params=None)

    X_test : npt.NDArray[np.float64] = X[n_train:]
    y_test : npt.NDArray[np.float64] = y[n_train:]
    X_test, y_test, params = standardize(X_test, y_test, train_set = False, params=params)

    max_vif, max_vif_idx = MaxVif(X_train)

    if (max_vif < 5):
        print(f"Max VIF: {max_vif.round(2)} at column: {int(max_vif_idx)}")
        print("Maximum VIF in training set < 5, no need to deal with multicollinearity")
    else:
        print(f"Max VIF: {max_vif.round(2)} at column: {int(max_vif_idx)}")
        raise ValueError("Maximum VIF in training set exceeds 5. Please address multicollinearity")
    
    return X_train, y_train, X_test, y_test


In [12]:
# solution based on normal equation

def NormalEquationSolution(X_train: npt.NDArray[np.float64], y_train : npt.NDArray[np.float64], X_test : npt.NDArray[np.float64], y_test : npt.NDArray[np.float64]):

    w_fit : npt.NDArray[np.float64] = normal_solution(X_train, y_train).reshape(-1)
    b_fit : float = 0.0
    # bias is zero after standardization, no need to fit it.
    print("--------------------------------------------------------")
    print("Solution based on Normal Equations")
    
    # performance metrics
    y_pred : npt.NDArray[np.float64] = np.zeros(y_test.shape)
    for i in range(len(y_pred)):
        y_pred[i] = f(X_test[i], w_fit, b_fit)

    perf : dict[str, float] = PerformanceSummary(y_test, y_pred)
    print("--------------------------------------------------------")
    print(f"MSE after training (test set): {perf['MSE'].round(3)}")
    print(f"MAE after training (test set): {perf['MAE'].round(3)}")
    print(f"R^2 after training (test set): {perf['R^2'].round(3)}")

    return None



In [13]:
# solution based on un-regularized gradient descent

def GradientDescentSolution(X_train : npt.NDArray[np.float64], y_train : npt.NDArray[np.float64], X_test : npt.NDArray[np.float64], y_test : npt.NDArray[np.float64], eta : float = 1e-3, eps : float = 1e-8):

    print("--------------------------------------------------------")
    print("Solution based on un-regularized Gradient Descent")
    print(f"Using eta = {eta}, eps = {eps}")

    w0 : npt.NDArray[np.float64] = np.random.randn(X_train.shape[-1])
    b0 : float = np.random.randn(1)

    print("--------------------------------------------------------")
    w_fit, b_fit = gd_solution(X_train, y_train, w0, b0, eta=eta, eps=eps)


    y_pred : npt.NDArray[np.float64] = np.zeros(y_test.shape)
    for i in range(len(y_pred)):
        y_pred[i] = f(X_test[i], w_fit, b_fit)
    perf : dict[str, float] = PerformanceSummary(y_test, y_pred)
    print("--------------------------------------------------------")
    print(f"MSE after training (test set): {perf['MSE'].round(3)}")
    print(f"MAE after training (test set): {perf['MAE'].round(3)}")
    print(f"R^2 after training (test set): {perf['R^2'].round(3)}")

    return None


In [14]:
# solution based on regularized gradient descent
def RegularizedGDSolution(X_train : npt.NDArray[np.float64], y_train : npt.NDArray[np.float64], X_test : npt.NDArray[np.float64], y_test : npt.NDArray[np.float64], lambd : float, regularization : str, eta : float = 1e-3, eps : float = 1e-8):

    print("--------------------------------------------------------")
    print("Solution based on Regularized Gradient Descent")
    print(f"Using lambda = {lambd}, regularization = {regularization}, eta = {eta}, eps = {eps}")

    w0 : npt.NDArray[np.float64] = np.random.randn(X_train.shape[-1])
    b0 : float = np.random.randn(1)


    print("--------------------------------------------------------")
    w_fit, b_fit = reg_gd_solution(X_train, y_train, w0, b0, lambd, regularization, eta=eta, eps=eps)


    y_pred : npt.NDArray[np.float64] = np.zeros(y_test.shape)
    for i in range(len(y_pred)):
        y_pred[i] = f(X_test[i], w_fit, b_fit)
    perf : dict[str , float] = PerformanceSummary(y_test, y_pred)

    print("--------------------------------------------------------")
    print(f"MSE after training (test set): {perf['MSE'].round(3)}")
    print(f"MAE after training (test set): {perf['MAE'].round(3)}")
    print(f"R^2 after training (test set): {perf['R^2'].round(3)}")
    
    return None

In [16]:
# putting it all together

n_samples = 1_000
n_train = int(0.8 * n_samples)
X_train, y_train, X_test, y_test = generate_datasets(n_samples, n_train, n_features = 10, collinear = True, n_collinear=1, corr_strength = 0.6, noise = 1.0)
NormalEquationSolution(X_train, y_train, X_test, y_test)
GradientDescentSolution(X_train, y_train, X_test, y_test, eta = 1e-3, eps = 1e-8)
RegularizedGDSolution(X_train, y_train, X_test, y_test, lambd = 0.01, regularization = 'ridge', eta = 1e-3, eps = 1e-8)
RegularizedGDSolution(X_train, y_train, X_test, y_test, lambd = 0.01, regularization = 'lasso', eta = 1e-3, eps = 1e-8)

Max VIF: 3.76 at column: 9
Maximum VIF in training set < 5, no need to deal with multicollinearity
--------------------------------------------------------
Solution based on Normal Equations
--------------------------------------------------------
MSE after training (test set): 0.055
MAE after training (test set): 0.188
R^2 after training (test set): 0.952
--------------------------------------------------------
Solution based on un-regularized Gradient Descent
Using eta = 0.001, eps = 1e-08
--------------------------------------------------------
Convergence achieved
--------------------------------------------------------
MSE after training (test set): 0.055
MAE after training (test set): 0.188
R^2 after training (test set): 0.951
--------------------------------------------------------
Solution based on Regularized Gradient Descent
Using lambda = 0.01, regularization = ridge, eta = 0.001, eps = 1e-08
--------------------------------------------------------
Convergence achieved
-----

## Conclusion

A common interpretation of linear regression is in terms of projecting the target vector $y$ onto the space spanned by the features $X$.  This is a very powerful interpretation, and allows us to define linear regression in general spaces, as long as they have an inner product defined on them. However, in this notebook, we have focused on the interpretation most common in machine learning, and one that is most directly relevant to the general ML framework - that in terms of a cost function being optimized.

We saw the implementation of Gradient Descent, and comparing it with the Normal Equations result (which is the analytical or "exact" solution), we see that it matches to the degree we care about. We also see that regularization did not help in our example. As mentioned before, regularization is primarily a technique to control model complexity, and thus, in the context of linear regression, primarily useful in solving for parameters when there are more features than examples.

In the upcoming notebooks, we shall discuss some more regression methods, namely, Principal Components Regression (PCR) and Partial Least Squares (PLS).